# Python による「スクレイピング & 自然言語処理」入門

## 1. はじめに
　クローラーとはWeb上のデータを自動的に収集するための道具です。クローラーを活用することで、担当者が手動で行っていたWeb情報収集の効率化、また自社だけでは入手できないさまざまなデータを取得し自社データと結合することで新たな示唆を得ることが可能になります。
 
　今回のセミナーでは初心者を対象にクローラーを作成し対象サイトのデータを収集、テキスト解析を行い、分析結果を得るまでの一連の流れについて、Python で使用するライブラリ、解析手法を交えて解説いたします。
 
 ※本発表は所属する組織とは一切関係がありません

　今回は対象ページとして、[日本酒物語 日本酒ランキング（人数）](http://www.sakeno.com/followrank/) とそれに紐づく各銘柄の詳細を収集し、各種分析を行います。本解析の内容として次の項目を含みます。

 * 解析のための下準備
   * 使用するライブラリのインストール
   * 使用するライブラリの読み込み
   * 定数の設定
 * クローラーによるデータの収集
   * ランキング一覧の生の HTML 確認
   * ランキング一覧のテーブル要素の取得
   * 詳細ページのデータ取得
 * テキスト解析
   * TFIDF によるレビュー中の特徴的な形容詞の抽出
   * 単語ベースのクラスタリング

## 2. クローラーとは？

（スライド参照）

## 3. 自然言語処理とは？

（スライド参照）

## 4. 解析のための下準備
### 4. 1 使用するライブラリのインストール

　まずは形態素解析ツール MeCab のインストールを行います。ここでは Mac (OSX Sierra) を仮定して進めています。

In [1]:
!brew install mecab mecab-ipadic
!pip install mecab-python3

　次にクローラー関係のライブラリもインストールします。ここでは以下の3つのライブラリを導入しています。

* html5lib
* requests
* BeautifulSoup

In [2]:
!conda install -y html5lib 
!conda install -y requests
!conda install -y BeautifulSoup4

Fetching package metadata .........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/tojima/anaconda3:
#
html5lib                  0.999                    py35_0  
Fetching package metadata .........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/tojima/anaconda3:
#
requests                  2.13.0                   py35_0  
Fetching package metadata .........
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /Users/tojima/anaconda3:
#
beautifulsoup4            4.5.3                    py35_0  


### 4.2. 使用するライブラリの読み込み

　ここではこの解析に使用する各種ライブラリを読み込んでいます。

In [3]:
# ファイル操作
import glob
import csv

# データ処理・視覚化
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# クローラー
import time
from datetime import datetime
from bs4 import BeautifulSoup
import requests

# テキスト解析
import MeCab
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering

### 4.3. 定数の設定

　ここでは取得対象となっているページのURLやクローラーの待ち時間、各種データの出力先を定義しています。

In [4]:
# 日本酒物語 日本酒ランキング（人数）の URL
FOLLOWRANK_URL = "http://www.sakeno.com/followrank/"

# クロール時の待ち時間
WAIT_TIME = 5

# 銘柄マスタの出力先
MEIGARA_MASTER_PATH = "../data/meigara_maseter.csv"
# 銘柄評価スコアの出力先ディレクトリ
MEIGARA_SCORES_DIR = "../data/meigara_scores/"
# 銘柄コメントの出力先ディレクトリ
MEIGARA_COMMENTS_DIR = "../data/meigara_comments/"

# TFIDF スコア算出後の結果出力先
TFIDF_PATH = "../data/tfidf.csv"
# クラスタリング結果の結果出力先
CLUSTER_PATH = "../data/cluster.csv"

## 5. クローラーによるデータの取得

　では、ここからクローラーでデータを取得するための流れを見ていきます。基本的な流れは次のようになります。
 
1. 対象ページのHTMLの取得。
2. 取得対象情報が含まれている部分のタグの特定。
3. 取得対象情報のパース。
4. 結果の保存。

では実際にクローラーのコードを見ていきましょう。

### 5.1 ランキング一覧の生の HTML 確認

　まずはランキングの一覧ページの HTML を取得します。ここでは `response` ライブラリを利用し取得します。

In [5]:
# ページの HTML を取得
response = requests.get(FOLLOWRANK_URL)

# 正しく取得できたかどうか HTTP ステータスコードで確認
if not response.status_code == 200:
    raise ValueError("Invalid response")
else:
    print("OK.")

OK.


正しくランキングページが取得されていれば `OK.` と出力されます。

次は取得してきた HTML の先頭 1000 件を見てみると…

In [6]:
response.text[:1000]

'<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\r\n<html lang="ja">\r\n<head>\r\n<meta http-equiv="Content-Type" content="text/html; charset=euc-jp">\r\n<meta http-equiv="Content-Style-Type" content="text/css">\r\n<meta http-equiv="Content-Script-Type" content="text/javascript">\r\n<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=4">\r\n<meta name="keywords" content="ÆüËÜ¼ò,¥é¥ó¥\xad¥ó¥°,¸ý¥³¥ß,É¾²Á">\r\n<title>ÆüËÜ¼ò¥é¥ó¥\xad¥ó¥°¡Ê¿Í¿ô¡Ë¡ÝÆüËÜ¼òÊª¸ì</title>\r\n<link rel="stylesheet" href="http://www.sakeno.com/incfiles/sakeno.css">\r\n<link rel="stylesheet" media="screen and (max-width:800px)" href="http://www.sakeno.com/incfiles/sakeno_sp.css">\r\n</head>\r\n\r\n<body><a id="top" name="top"></a>\r\n<div id="bigbox">\r\n\r\n\t<div id="header"><div id="headertitle"><h1><a href="http://www.sakeno.com/"><img border="0" src="http://www.sakeno.com/images/logo_new.gif" width="246" height="54" alt="ÆüËÜ¼

このように対象ページの HTML が文字列として取得できたが、文字化けしている。文字列の先頭の方を見てみると、`charset=euc-jp`という文字列が見えるので、euc-jpということを考慮して扱ってみる。

In [7]:
response.encoding = 'euc_jp'
print(response.text[:1000])

<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">
<html lang="ja">
<head>
<meta http-equiv="Content-Type" content="text/html; charset=euc-jp">
<meta http-equiv="Content-Style-Type" content="text/css">
<meta http-equiv="Content-Script-Type" content="text/javascript">
<meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=4">
<meta name="keywords" content="日本酒,ランキング,口コミ,評価">
<title>日本酒ランキング（人数）−日本酒物語</title>
<link rel="stylesheet" href="http://www.sakeno.com/incfiles/sakeno.css">
<link rel="stylesheet" media="screen and (max-width:800px)" href="http://www.sakeno.com/incfiles/sakeno_sp.css">
</head>

<body><a id="top" name="top"></a>
<div id="bigbox">

	<div id="header"><div id="headertitle"><h1><a href="http://www.sakeno.com/"><img border="0" src="http://www.sakeno.com/images/logo_new.gif" width="246" height="54" alt="日本酒物語"></a></h1></div><div id="headerlogin"><a href="http://www.sakeno.com/l


正しく出力された。

### 5.2 ランキング一覧のテーブル要素の取得

　今度はこのページから `find` メソッドを利用し、 `<table> 〜 </table>` の部分を抜き出します。この処理には `BeautifulSoup` を利用します。

In [8]:
soup = BeautifulSoup(response.text, "lxml")
table = soup.body.find("table")

　次はテーブルの中の行要素を全て取得します。`<tr> 〜 </tr>` の部分が各行に該当します。`find_all`メソッドを利用すれば各行が1要素となったリスト構造として取得可能です。

In [9]:
trs = table.find_all("tr")[2:] # 先頭のゴミをカット

各行から必要な数値、文字列を抜き出します。

In [10]:
# ある行の情報をパースし以下の要素を取得する。
#
#   [ 順位, 銘柄, 銘柄の読み, 
#     蔵元, 蔵元の県, 蔵元の市町村,
#     銘柄詳細ページのURL ]
#
def parse_tr(tr):
    # 順位
    tds = tr.find_all("td")
    rank = int(tds[0].get_text().split("位")[0]) 

    # 銘柄
    a = tds[1].find("a")
    meigara = a.get_text()
    detail_url = a.get("href")
    yomi = tds[1].find("div").string

    # 酒造
    location = tds[2].find_all("a")
    kuramoto = location[0].string
    prefecture = location[1].string
    city = location[2].string
    
    tr_l = [
        rank, meigara, yomi,
        kuramoto, prefecture, city,
        detail_url
    ]
    return tr_l


ranking_list = [parse_tr(tr) for tr in trs]
ranking_list[:5]

[[1,
  '獺祭',
  'だっさい',
  '旭酒造（山口県）',
  '山口県',
  '岩国市',
  'http://www.sakeno.com/meigara/931'],
 [2,
  '醸し人九平次',
  'かもしびとくへいじ',
  '萬乗醸造',
  '愛知県',
  '名古屋市',
  'http://www.sakeno.com/meigara/735'],
 [3,
  '出羽桜',
  'でわざくら',
  '出羽桜酒造',
  '山形県',
  '天童市',
  'http://www.sakeno.com/meigara/219'],
 [4, '田酒', 'でんしゅ', '西田酒造店', '青森県', '青森市', 'http://www.sakeno.com/meigara/11'],
 [5, '黒龍', 'こくりゅう', '黒龍酒造', '福井県', '吉田郡', 'http://www.sakeno.com/meigara/667']]

あとの処理のためにデータフレームに変換します。この際にユニークな連番IDを付加します。

In [11]:
meigara_master_df = pd.DataFrame(
    ranking_list,
    columns=["rank", "meigara", "yomi",
             "kuramoto", "prefecture", "city",
             "detail_url"]
)

# ユニークな連番 ID を追加
meigara_master_df["meigara_id"] = meigara_master_df.index.to_series() + 1
meigara_master_df = meigara_master_df[
    ["meigara_id",
     "rank", "meigara", "yomi",
     "kuramoto", "prefecture", "city",
     "detail_url"]
]

# 銘柄マスタデータの出力
meigara_master_df.to_csv(
    MEIGARA_MASTER_PATH,
    encoding="utf-8",
    sep=",",
    index=False
)

meigara_master_df.head()

,meigara_id,rank,meigara,yomi,kuramoto,prefecture,city,detail_url
0,1,1,獺祭,だっさい,旭酒造（山口県）,山口県,岩国市,http://www.sakeno.com/meigara/931
1,2,2,醸し人九平次,かもしびとくへいじ,萬乗醸造,愛知県,名古屋市,http://www.sakeno.com/meigara/735
2,3,3,出羽桜,でわざくら,出羽桜酒造,山形県,天童市,http://www.sakeno.com/meigara/219
3,4,4,田酒,でんしゅ,西田酒造店,青森県,青森市,http://www.sakeno.com/meigara/11
4,5,5,黒龍,こくりゅう,黒龍酒造,福井県,吉田郡,http://www.sakeno.com/meigara/667


ここまでで、ランキング一覧の結果が無事取得できました。

### 5.3 詳細ページのデータ取得

　次は先程取得したランキング一覧のデータを利用して、詳細ページ（`detail_url`）から以下の情報を取得します。
 * 数値による評価データ (良い／悪い)
   * 味
   * 香り
   * 濃さ
   * 価格
   * デザイン
 * コメント一覧
   * 投稿ID
   * タイトル
   * 投稿日時
   * ユーザ名
   * テキスト

これらのデータを集めるために、以下に次の3つの関数を記述しています。

* 数値データによる評価データ取得用の関数
* コメント一覧取得用の関数
* すべての詳細ページからデータを取得するための関数

In [12]:
# 数値による評価データ取得関数
def parse_scores_table(soup, meigara_id):
    scores_table = soup.body.find_all("form")[1]
    trs = scores_table.find_all("tr")

    # 味
    aji = trs[2].find_all("span")
    aji_good = int(aji[0].string)
    aji_bad = int(aji[1].string)

    # 香り
    kaori = trs[3].find_all("span")
    kaori_good = int(kaori[0].string)
    kaori_bad = int(kaori[1].string)

    # 濃さ
    kosa = trs[4].find_all("span")
    kosa_good = int(kosa[0].string)
    kosa_bad = int(kosa[1].string)

    # 価格
    kakaku = trs[5].find_all("span")
    kakaku_good = int(kakaku[0].string)
    kakaku_bad = int(kakaku[1].string)

    # デザイン
    design = trs[6].find_all("span")
    design_good = int(design[0].string)
    design_bad = int(design[1].string)

    score_li = [
        [meigara_id, "味", aji_good, aji_bad],
        [meigara_id, "香り", kaori_good, kaori_bad],
        [meigara_id, "濃さ", kosa_good, kosa_bad],
        [meigara_id, "価格", kakaku_good, kakaku_bad],
        [meigara_id, "デザイン", design_good, design_bad]
    ]

    score_df = pd.DataFrame(
        score_li,
        columns=["meigara_id", "name", "good_score", "bad_score"]
    )
    
    # (index)  name   good_score bad_score
    #       0  味           1123       260
    #       1  香り         1095       250
    #       2  濃さ          978       304
    #       3  価格          950       344
    #       4  デザイン       975       249
    
    return score_df

In [13]:
# コメント一覧取得関数
def parse_comments_table(soup, meigara_id):
    reviews_table = soup.body.find_all("table")[-1]

    # 以下のような構造になっているため、dtのみ、ddのみで処理し、
    #  最後に concat で横方向に単純結合する
    #
    #   <dt>〜</dt><dd>〜</dd>
    #   <dt>〜</dt><dd>〜</dd>
    #   <dt>〜</dt><dd>〜</dd>
    #   ...
    #

    # <dt>〜</dt> の処理
    dts = [
        [meigara_id,
         int(dt.contents[0].get("name").replace("voice", "")),
         dt.contents[3].string]
        for dt
        in reviews_table.find_all("dt")
    ]
    dts_df = pd.DataFrame(
        dts,
        columns=["meigara_id", "toukou_id", "title"]
    )
    
    # <dd>〜</dd> の処理
    dds = [
        [dd.contents[-1].text.split("（")[1].split("）")[0],
         dd.contents[-1].find("a").string,
         dd.contents[-2].replace("\n", " ")]
        for dd
        in reviews_table.find_all("dd")
    ]
    dds_df = pd.DataFrame(
        dds,
        columns=["created_at", "user_name", "text"]
    )
    dds_df["created_at"] = dds_df["created_at"].apply(
        lambda x: datetime.strptime(x, '%Y年%m月%d日 %H時%M分%S秒')
        )
    
    # 結合
    comments_df = pd.concat([dts_df, dds_df], axis=1)
    
    # (index) meigara_id toukou_id title              created_at           user_name  text
    #       0          1      6193 すっきりして飲みやすい 2016-10-20 11:57:54  あいうそん   おいしいです
    #   ...

    return comments_df

In [14]:
# すべての詳細ページからデータを取得するための関数
def parse_maigara_detail_page(row):
    print(
        datetime.now().isoformat(sep=" "),
        row["meigara_id"],
        row["meigara"]
    )
    
    # 連続アクセス時の負荷軽減
    time.sleep(WAIT_TIME)
    
    # クローリング
    response = requests.get(row["detail_url"])
    if not response.status_code == 200:
        raise ValueError("Invalid response")
    response.encoding = 'euc_jp'
    # ゴミとなる文字群を除去
    preprocessed_html_string = response.text.replace("<br>", "\n")
    preprocessed_html_string = preprocessed_html_string.replace("\r", "")
    preprocessed_html_string = preprocessed_html_string.replace("　", " ")
    soup = BeautifulSoup(preprocessed_html_string, "lxml")
    
    # 評価スコアの取得 & 出力
    score_df = parse_scores_table(soup, row["meigara_id"])
    scores_path = MEIGARA_SCORES_DIR + str(row["meigara_id"]) + ".csv"
    score_df.to_csv(
        scores_path,
        encoding="utf-8",
        sep=",",
        index=False,
        quoting=csv.QUOTE_NONNUMERIC
    )
    
    # 評価コメントの取得 & 出力
    comments_df = parse_comments_table(soup, row["meigara_id"])
    comments_path = MEIGARA_COMMENTS_DIR + str(row["meigara_id"]) + ".csv"
    comments_df.to_csv(
        comments_path,
        encoding="utf-8",
        sep=",",
        index=False,
        quoting=csv.QUOTE_NONNUMERIC
    )
    return

次にこれらのコードを全ての銘柄に対して実行します。

In [15]:
for idx, row in meigara_master_df.iterrows():
    parse_maigara_detail_page(row)

2017-03-12 17:58:02.529907 1 獺祭
2017-03-12 17:58:09.126493 2 醸し人九平次
2017-03-12 17:58:15.797209 3 出羽桜
2017-03-12 17:58:22.873022 4 田酒
2017-03-12 17:58:29.302859 5 黒龍
2017-03-12 17:58:35.890476 6 飛露喜
2017-03-12 17:58:42.338474 7 新政
2017-03-12 17:58:49.018110 8 雪の茅舎
2017-03-12 17:58:55.609204 9 鳳凰美田
2017-03-12 17:59:01.490036 10 風の森
2017-03-12 17:59:07.875526 11 鍋島
2017-03-12 17:59:14.489016 12 くどき上手
2017-03-12 17:59:20.826484 13 十四代
2017-03-12 17:59:27.388108 14 菊姫
2017-03-12 17:59:34.022284 15 天狗舞
2017-03-12 17:59:40.585017 16 神亀
2017-03-12 17:59:46.673921 17 浦霞
2017-03-12 17:59:52.898387 18 鶴齢
2017-03-12 17:59:59.212296 19 楯野川
2017-03-12 18:00:05.593475 20 八海山
2017-03-12 18:00:12.830346 21 雁木
2017-03-12 18:00:19.296840 22 開運
2017-03-12 18:00:25.956883 23 大七
2017-03-12 18:00:32.476801 24 〆張鶴
2017-03-12 18:00:38.731137 25 久保田
2017-03-12 18:00:45.247792 26 酔鯨
2017-03-12 18:00:51.643924 27 手取川
2017-03-12 18:00:58.161813 28 東一
2017-03-12 18:01:07.472091 29 陸奥八仙
2017-03-12 18:01:14.158923 30

ここまでで、銘柄のマスタデータ、詳細ページの評価、詳細ページのコメントの情報が手に入りました。

## 6. テキスト解析

　ここからは先程クローラーで収集したデータを利用して、TFIDFによるレビュー中の特徴的な形容詞の抽出と単語ベースのクラスタリングを行っていきます。

### 6.1 TFIDF によるレビュー中の特徴的な形容詞の抽出

　この解析では、あるドキュメント中における特徴的な単語（特徴語）の抽出を行います。集めたデータを各種統計処理で扱えるようにするためには、行列形式に変換する必要があります。今回は Bag-of-Words モデルを用いて、単語を行列の形に変換します。Bag-of-Words とは、文章に単語が含まれているかどうかのみを考え、単語の並び方などは考慮しないモデルのことです。一番シンプルなモデルは単語があれば 1、なければ 0 となります。また、単語の出現回数をそのまま使う (Term Frequency) という方法もあります。これは文書中にある単語が含まれている回数をそのまま値として用います。

```
すもももももももものうち  (1)
↓
[すもも, も, もも, も, もも, の, うち]  (2)
↓
{すもも: 1, も:2, もも: 2, の: 1, うち:1}  (3)
```

　そして各ドキュメントに含まれる単語を列に、文書を行とすると単語の出現回数を要素とした行列形式に変換できます。例として、 (a) 「すもももももももものうち」、(b) 「料理も景色もすばらしい」、(c) 「私の趣味は写真撮影です」という3つの文書を考えます。列のラベルは単語の出現の早い順に [すもも, も, もも, の, うち, 料理, 景色, 素晴らしい, 私, 趣味, は, 写真撮影, です] とすると、文書行列は下記のようになります。

```
[[1,2,2,1,1,0,0,0,0,0,0,0,0],    #  (a)
 [0,2,0,0,0,1,1,1,0,0,0,0,0],    #  (b)
 [0,0,0,1,0,0,0,0,1,1,1,1,1]]    #  (c)
```
 
　今回は数値に Term Frequency を用います。この変換を行うためには、元のテキストデータを単語単位に分割する必要があります。これを行うためには形態素解析ツールのMeCabを利用します。
 
　次に特徴量の計算には TFIDF を用います。TFIDF は TF と IDF というの2つの値を掛けあわせた指標のことです。TF は文書内における単語の出現頻度を表します。これは「ある文書中である単語が何回出現したか」で定義されます。1つの文書に多く出現する単語ほど重要度が高くなります。IDF は多数の文書に出現する単語ほど重要度が低くなるようなスコアです。「ある単語が含まれている文書数を全ての文書数で割ったものの逆数」で定義されます。つまり、TFIDF が大きな値になるということは、「文書内である特定の単語が多く出現し、かつその単語は他の文書ではほとんど出現しない」ということを表します。例えば、「私」という単語は、各文書内における出現回数は多いですが、多くの文書に出現するので重要度は下がります。逆に「特許」という単語は、「特許」を話題の中心にしている特定の文書には文書内には多く現れ、一般的な文書には現れない単語なので重要度は上がります。

　これらの解析を行うためのコードを見ていきましょう。まずは単語の分割を行うための関数を定義します。ここでは対象の品詞のみに絞り込んで、原形のみを抽出するようにしています。

In [16]:
def split_text(text, target_pos=["形容詞"]):
    tagger = MeCab.Tagger()
    text_str = text
    tagger.parse('')
    node = tagger.parseToNode(text_str)

    words = []
    while node:
        l = node.feature.split(",")
        pos = l[0]
        if pos in target_pos:
            # unicode 型に戻す
            if l[6] == "*":
                word = node.surface # 変化しない語は表層形をそのまま使う
            else:
                word = l[6]         # 動詞や形容詞は原形を使う
            words.append(word)
        node = node.next
    return " ".join(words)          # スペース区切りで単語を結合し返す

　では実際に全銘柄のコメントを読み込んで、単語単位に分割し必要な単語のみを取り出してみましょう。

In [17]:
# 全ファイル読み込み
comment_files = glob.glob("../data/meigara_comments/*.csv")

# 縦方向に単純結合
comment_df = pd.concat([pd.read_csv(f) for f in comment_files])
comment_df = comment_df.reset_index()

# 全ての text に対して形容詞の抽出を行う
comment_df["split"] = comment_df["text"].map(split_text)
comment_df.head()

,index,meigara_id,toukou_id,title,created_at,user_name,text,split
0,0,1,6193,すっきりして飲みやすい,2016-10-20 11:57:54,あいうそん,おいしいです,おいしい
1,1,1,6126,獺祭 等外２３,2016-08-08 22:20:28,富牟谷欠,獺祭 等外２３ 山田錦２３ 生酒 ２７ＢＹ ライチ様な立ち香、含み香。抜ける香りはやや甘く。...,甘い 強い 淡い ない 濃い
2,2,1,5992,獺祭50,2016-04-26 19:24:02,季がらし,獺祭と言えば高精米、磨きが強調され、50%はその最低ランクである しかし全国の銘酒蔵もこの5...,美味い 美味い
3,3,1,5946,獺祭等外,2016-03-18 20:37:29,季がらし,旭酒造蔵本の売店で買った普通酒！ 山田錦は栽培時、5%以上の等外米(規格外)が出てしまい純米...,美味しい 悪い
4,4,1,5940,スパークリング、うすにごり,2016-03-13 23:16:16,nomuyoshi,抜栓直後、瓶から上る香りは若々しく青いような香り。 上る香りはさっぱりとした果物のよう。 口...,若々しい 青い 鋭い


銘柄別に全単語を結合します。

In [18]:
meigara_comments_df = comment_df\
    .groupby("meigara_id")["split"]\
    .apply(lambda x: "%s" % ' '.join(x))\
    .reset_index()
meigara_comments_df.head()

,meigara_id,split
0,1,おいしい 甘い 強い 淡い ない 濃い 美味い 美味い 美味しい 悪い 若々しい 青い 鋭い...
1,2,ない ない 美味い 美味い ない 早い 早い 若い 美味い 高い たまらない いい 甘い ...
2,3,甘い 柔らかい 無い 美味い イイ やすい 力強い 鋭い 美味い 甘い くどい 旨い 甘い ...
3,4,いい 若い 深い 無い 新しい 甘い 濃い 旨い 美味い 高い 物足りない 甘酸っぱい 良...
4,5,軽い 良い ほしい 強い うまい 不味い 苦い 良い 良い 広い 美味しい ない 美味しい...


　今回は数値に Term Frequency を用います。scikit-learn に [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)というものがあり、単語と列番号の対応付けなどの作業をまとめて行うことが出来ます。この次に TFIDF の計算を行う場合、さらに簡易化したライブラリが存在します。
 
　TFIDF の計算は scikit-learn の [TfidfVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html) を用いれば、前述の CountVectorizer による行列化と TFIDF の計算を同時に行うことが出来るので、今回はこれを用いて TFIDF の計算を行います。では実際に TfidfVectorizer を利用して、TFIDF の計算を行ってみましょう。

In [19]:
vectorizer = TfidfVectorizer()
tfidfs = vectorizer.fit_transform(meigara_comments_df["split"])
tfidfs

<200x292 sparse matrix of type '<class 'numpy.float64'>'
	with 4099 stored elements in Compressed Sparse Row format>

では計算結果から、各銘柄におけるスコアの上位から5単語ずつ取り出してみましょう。

In [20]:
## TFIDF の結果からi 番目のドキュメントの特徴的な上位 n 語を取り出す
def extract_feature_words(terms, tfidfs, i, n):
    tfidf_array = tfidfs[i]
    top_n_idx = tfidf_array.argsort()[-n:][::-1]
    words = [terms[idx] for idx in top_n_idx]
    return words

In [21]:
# index 順の単語のリスト
terms = vectorizer.get_feature_names()

In [22]:
top_n = 5  # 上位5件
highscore_words = [
    extract_feature_words(terms, tfidfs.toarray(), i, top_n)
    for i
    in range(len(meigara_comments_df.index))
]
highscore_words_str = [" ".join(l) for l in highscore_words]
meigara_comments_df["highscore_words"] = highscore_words_str
meigara_comments_df.head()

,meigara_id,split,highscore_words
0,1,おいしい 甘い 強い 淡い ない 濃い 美味い 美味い 美味しい 悪い 若々しい 青い 鋭い...,良い 美味しい 悪い ない うまい
1,2,ない ない 美味い 美味い ない 早い 早い 若い 美味い 高い たまらない いい 甘い ...,美味い 不味い ない 甘い 旨い
2,3,甘い 柔らかい 無い 美味い イイ やすい 力強い 鋭い 美味い 甘い くどい 旨い 甘い ...,素晴らしい 甘い やすい イイ 美味しい
3,4,いい 若い 深い 無い 新しい 甘い 濃い 旨い 美味い 高い 物足りない 甘酸っぱい 良...,良い 旨い 無い うまい ない
4,5,軽い 良い ほしい 強い うまい 不味い 苦い 良い 良い 広い 美味しい ない 美味しい...,良い うまい おいしい ない いい


銘柄マスタと結合して、結果を確認してみましょう。

In [23]:
# マスタデータの JOIN
master_df = pd.read_csv(MEIGARA_MASTER_PATH)
result_df = master_df.merge(meigara_comments_df, on="meigara_id", how="inner")
result_df.head()

,meigara_id,rank,meigara,yomi,kuramoto,prefecture,city,detail_url,split,highscore_words
0,1,1,獺祭,だっさい,旭酒造（山口県）,山口県,岩国市,http://www.sakeno.com/meigara/931,おいしい 甘い 強い 淡い ない 濃い 美味い 美味い 美味しい 悪い 若々しい 青い 鋭い...,良い 美味しい 悪い ない うまい
1,2,2,醸し人九平次,かもしびとくへいじ,萬乗醸造,愛知県,名古屋市,http://www.sakeno.com/meigara/735,ない ない 美味い 美味い ない 早い 早い 若い 美味い 高い たまらない いい 甘い ...,美味い 不味い ない 甘い 旨い
2,3,3,出羽桜,でわざくら,出羽桜酒造,山形県,天童市,http://www.sakeno.com/meigara/219,甘い 柔らかい 無い 美味い イイ やすい 力強い 鋭い 美味い 甘い くどい 旨い 甘い ...,素晴らしい 甘い やすい イイ 美味しい
3,4,4,田酒,でんしゅ,西田酒造店,青森県,青森市,http://www.sakeno.com/meigara/11,いい 若い 深い 無い 新しい 甘い 濃い 旨い 美味い 高い 物足りない 甘酸っぱい 良...,良い 旨い 無い うまい ない
4,5,5,黒龍,こくりゅう,黒龍酒造,福井県,吉田郡,http://www.sakeno.com/meigara/667,軽い 良い ほしい 強い うまい 不味い 苦い 良い 良い 広い 美味しい ない 美味しい...,良い うまい おいしい ない いい


最後に結果をCSVファイルとして出力します。

In [24]:
target_cols = ["rank", "meigara", "kuramoto", "detail_url", "highscore_words"]
result_df[target_cols].to_csv(TFIDF_PATH, index=False)

ここまでが特徴語抽出を行うまでの一連の流れとなります。

### 6.2 単語ベースのクラスタリング

　次は評価が似た銘柄同士をまとめる方法を見ていきたいと思います。サンプルデータが大量にある場合、似た者同士をまとめることで、新たな知見が得られる可能性があります。ここではその一連の流れを見ていきます。

　まず最初行わなければならないのは、特徴語の抽出の場合と同じく各文書に含まれる単語を行列形式に変換することです。今回も Term Frequency を用いた Bag-of-Words モデルで変換します。
 
　次にクラスタリングを行う前に、行列に対していくつか前処理を行う必要があります。まずレビューの件数が大きく異なるので、数値を標準化してやる必要があります。今回は「Zスコア」という標準化を行います。これは各要素から平均を引いて、標準偏差で割ったものです。この変換を行うと、平均が 0 で標準偏差・分散が 1 になります。この変換を行うためのライブラリとして scikit-learn には StandardScaler があります。また、行列は疎な状態となっています。このような場合は次元圧縮を行うことで、より効率が良く、直感に近いクラスタリング結果を得られます。今回は[主成分分析：PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html)を利用して次元を圧縮しています。
 
　最後にクラスタリングを行います。今回はコサイン類似度を距離の基準とした階層型クラスタリングを行いクラスタを決定しています。
 

　では実際のコードを確認していきましょう。まずは対象の単語の抽出です。今回は名詞、動詞、形容詞を抽出しています。
 

In [25]:
noun_verb_adj_words = comment_df["text"]\
    .apply(lambda x: split_text(x, target_pos=["名詞", "動詞", "形容詞"]))
comment_df["split_noun_verb_adj"] = noun_verb_adj_words
meigara_comments_df = comment_df\
    .groupby("meigara_id")["split_noun_verb_adj"]\
    .apply(lambda x: "%s" % ' '.join(x))\
    .reset_index()

meigara_comments_df.head()

,meigara_id,split_noun_verb_adj
0,1,おいしい 獺 祭 等外 ２ ３ 山田 錦 ２ ３ 生酒 ２ ７ ＢＹ ライチ 様 立ち 香 ...
1,2,Wow very oishii Sake ! 米 吟醸 aka 醸す 人 九 平次 彼 地 ...
2,3,上 立つ 仄か 甘い 香り 含み 柔らかい 入る 派手 さ 無い 純大 吟 旨み 特徴 個 ...
3,4,好き 酒 一つ する コク ある いい 感じ 田 酒 特 純生 飲む 開 栓 直後 含む す...
4,5,甘み 感じる する 辛口 ｡ 後味 軽い 酸味 ある ｡ ラベル 飲む 方 ｢ 冷やす ｣ ...


次に行列への変換です。前述の CountVectorizer を利用することで、簡単に変換できます。

In [26]:
vectorizer = CountVectorizer()
word_counts = vectorizer.fit_transform(meigara_comments_df.split_noun_verb_adj)
wca = word_counts.toarray()
wca

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

StandardScaler を利用することで標準化も簡単に行えます（int 型から float 型への変換は意図通りなので警告内容については問題ない）。

In [27]:
sds = StandardScaler()
X = sds.fit_transform(wca)

/Users/tojima/anaconda3/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


次は次元圧縮を行います。今回は30次元に圧縮しています。この圧縮された行列の各行は、それぞれの文書が表す概念を表現しているものとなり、概念ベクトルと呼べるものとなります。

In [28]:
model = PCA(n_components=30)
X_decomp = model.fit_transform(X)
X_decomp

array([[  2.28167681e+02,  -1.16255034e+02,  -4.89383813e+01, ...,
          9.14023707e-02,  -8.60942515e-01,  -6.03637929e-01],
       [  1.12847840e+02,   1.94123781e+02,  -6.26221348e+01, ...,
          3.46364954e-01,  -1.24565097e+00,  -6.24311590e-01],
       [  2.25193159e+01,   6.46982151e+00,   1.69020459e+01, ...,
          1.00232518e+00,   6.03836210e-02,  -5.87677278e-02],
       ..., 
       [ -8.47630942e+00,  -2.13891471e+00,  -4.14508268e+00, ...,
         -8.44277795e-02,  -4.82077079e-01,  -4.45857641e-01],
       [ -4.62522339e+00,  -3.79801541e-01,  -3.06852519e+00, ...,
          4.29486114e-01,  -1.49074427e+00,   2.41093999e-01],
       [ -6.23976436e+00,  -1.28528158e+00,  -2.87711006e+00, ...,
          1.53063625e-01,   9.22628033e-02,  -9.44972731e-01]])

さて、ここまでで下準備が終わったのでクラスタリングを実行しましょう。今回はコサイン類似度を基準とした6つのクラスタに分けています。

In [29]:
model = AgglomerativeClustering(n_clusters=6, linkage="average", affinity="cosine")
y = model.fit_predict(X_decomp)
y

array([2, 2, 2, 2, 2, 5, 0, 2, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 0, 1, 3, 0, 0,
       1, 2, 3, 3, 3, 3, 0, 0, 0, 3, 0, 2, 0, 2, 1, 3, 3, 2, 1, 3, 3, 3, 0,
       0, 3, 3, 3, 3, 3, 3, 3, 1, 2, 0, 3, 3, 3, 3, 0, 0, 3, 2, 3, 3, 3, 3,
       3, 3, 0, 0, 3, 2, 3, 3, 0, 3, 2, 3, 3, 3, 3, 3, 0, 4, 3, 2, 3, 3, 3,
       3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 2, 3, 3, 3, 3, 3, 0, 3, 3,
       3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 0, 3, 3, 3, 3,
       3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3,
       2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2,
       3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3])

クラスタ番号が算出できたので、銘柄マスタデータと結合し結果を確認してみましょう。

In [30]:
# マスタデータの JOIN
master_df = pd.read_csv(MEIGARA_MASTER_PATH)
result_df = master_df.merge(meigara_comments_df, on="meigara_id", how="inner")

result_df["cluster"] = y
result_df.sort_values(by=["cluster", "rank"])

,meigara_id,rank,meigara,yomi,kuramoto,prefecture,city,detail_url,split_noun_verb_adj,cluster
6,7,7,新政,あらまさ,新政酒造,秋田県,秋田市,http://www.sakeno.com/meigara/55,冷蔵 する いる 気 つける 開 栓 する プシュー 吹き出す しまう 半年 寝かせる いる...,0
8,9,7,鳳凰美田,ほうおうびでん,小林酒造（栃木県）,栃木県,小山市,http://www.sakeno.com/meigara/95,鳳凰 美田 濾過 本 生 ２ ０ １ ７ / ２ 頂く アル 添 フルーティー 香り ラベル...,0
9,10,10,風の森,かぜのもり,油長酒造,奈良県,御所市,http://www.sakeno.com/meigara/898,風 森 ALPHA TYPE 3 米 吟醸 八 錦 ５ ０ ２ ７ ＢＹ 軽い 吟醸 香 含...,0
10,11,10,鍋島,なべしま,富久千代酒造,佐賀県,鹿島市,http://www.sakeno.com/meigara/1482,春先 購入 する 半年 寝かせる もの タッチ チリ する 白 ワイン 的 ブドウ シャープ...,0
16,17,17,浦霞,うらかすみ,佐浦,宮城県,塩竈市,http://www.sakeno.com/meigara/47,素っ気 ラベル 純 米 事 原酒 事 わかる メチャクチャ ぶっきらぼう 強面 塩釜 漁師 ...,0
17,18,18,鶴齢,かくれい,青木酒造（新潟県）,新潟県,南魚沼市,http://www.sakeno.com/meigara/583,鶴 齢 特別 米 越 淡い 麗 ５ ５ ％ 濾過 生 原酒 ２ ８ ＢＹ 綺麗 果実 香 含...,0
18,19,18,楯野川,たてのかわ,楯の川酒造,山形県,酒田市,http://www.sakeno.com/meigara/229,楯 野川 純 米 吟醸 山田 ５ ０ 汲む 夏 熟 ２ ７ ＢＹ 好い 熟れる 甘い 果実 ...,0
21,22,21,開運,かいうん,土井酒造場,静岡県,掛川市,http://www.sakeno.com/meigara/729,酒屋 特 本 思う 特 純 特 純 包み 紙 オレンジ ん 思い返す 飲 購入 する みる ...,0
22,23,21,大七,だいしち,大七酒造,福島県,二本松市,http://www.sakeno.com/meigara/381,全体 的 濃い 醇 いう もと 生まれる コシ 力強い さ ある いい ん じん わり 感じ...,0
29,30,30,写楽（寫樂）,しゃらく,宮泉銘醸,福島県,会津若松市,http://www.sakeno.com/meigara/1804,一 回 火入れ 米 酒 バランス いい 言う の 特徴 感じる られる の 甘い さ 旨み ...,0


最後に結果を出力します。

In [31]:
target_cols = ["rank", "meigara", "kuramoto", "detail_url", "cluster"]
result_df[target_cols].to_csv(CLUSTER_PATH, index=False)

このような流れで単語ベースのクラスタリングが行えます。

## 7. おわりに

　今回の解析はまだまだ不足している点があります。例えば以下のような点を考慮しませんでした。
 
 * 形態素解析用辞書の改善
   * デフォルトのままだと例えば「山田錦」が「山田」＋「錦」に分割されてしまう。
 * 否定語の扱い
   * 美味しくない → 美味しい ＋ ない と分割され、このままでは「美味しい」としてカウントされてしまう。
 * 数値を含んだ単語の取扱
   * 例えばアルコール度数を表すような数値や、精米歩合の数値などがうまく扱えていない。
 * ノイズとなるような単語のカット
   * 「Wow very oishii Sake ! 」などのテキストが含まれているが、このようなものが含まれていてもあまり有益な結果をえられないので、カットすべき。
 
これらの点などを改善していくことにより、より我々の感覚と近い結果を得られるようになります。

In [36]:
len(vectorizer.get_feature_names())#wca[0])

7252